In [1]:
import pandas as pd

TASKNAMES = ["subtask1", "subtask2"]
BATCHNAMES = ["batch1", "batch2", "batch3"]

dataset_loader = dict()

for taskname in TASKNAMES:
    for batchname in BATCHNAMES:
        print(f"Processing {taskname} for {batchname}")
        prefix = "dataset/"
        # Load the data
        
        if batchname == "batch2" and taskname == "subtask1":
            data = pd.read_csv(f"{prefix}/{taskname}_train_{batchname}_preprocess.csv")
        else:
            data = pd.read_csv(f"{prefix}/{taskname}_train_{batchname}.csv")
        dataset_loader[f"{taskname}_{batchname}"] = data
        

Processing subtask1 for batch1
Processing subtask1 for batch2
Processing subtask1 for batch3
Processing subtask2 for batch1
Processing subtask2 for batch2
Processing subtask2 for batch3


### Statistics

Hereby I list the number of each unique answers and claims for each batch datatset

| Task | Batch | number of answers | number of claims |
| ---  | ----- | ----------------- | ---------------- |
| 1    | 1     | 235               | 500              |
| 1    | 2     | 468               | 2092             |
| 1    | 3     | 469               | 1500             |


(seems that subtask 1 and 2 have the shared dataset)
| Task | Batch | number of answers | number of claims |
| ---  | ----- | ----------------- | ---------------- |
| 2    | 1     | 235               | 500              |
| 2    | 2     | 468               | 2092             |
| 2    | 3     | 469               | 1500             |

# Testing to get named entity in reference

Run `process_text_to_knowledge_graph` function on reference with *gemini-2.5-flash* model.

Now we use gemini-2.5 flash model and you can use it for free. Register at google vertext AI and create an API key, put it in `.env` file in the same folder with `GEMINI_API_KEY=...`.

In [2]:
import asyncio
import os
import networkx as nx

from dotenv import load_dotenv
from operationCheatSheet import chunking_by_token_size, extract_entities, extract_evidence
from utils import compute_mdhash_id
from geminillm import gemini_complete_if_cache
from openaillm import openai_embed, openai_complete
from faiss_impl import FaissVectorDBStorage
from networkx_impl import NetworkXStorage
from json_kv_iml import JsonKVStorage
from shared_storage import initialize_share_data
from cheatsheet import CHEATSHEETS


# LLM_MODEL_NAME = "gpt-3.5-turbo"
LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20"

load_dotenv()

async def process_text_to_knowledge_graph_on_claim(text_content,
                                          cheatsheet_knowledge_graph_inst,
                                          max_gleaning = 1,
                                          file_path="unknown_source",
                                          prefix=""):
    # Step 1: Create chunks from your text content
    raw_chunks = chunking_by_token_size(
        content=text_content,
        split_by_character=None,  # Optional: specify a character to split by (e.g., "\n\n")
        split_by_character_only=False,
        overlap_token_size=128,
        max_token_size=1024,
        tiktoken_model=LLM_MODEL_NAME
    )
    
    # Step 2: Format chunks as required by extract_entities
    doc_id = compute_mdhash_id(text_content, prefix="doc-")
    chunks = {
        compute_mdhash_id(chunk["content"], prefix="chunk-"): {
            **chunk,
            "full_doc_id": doc_id,
            "file_path": file_path
        }
        for chunk in raw_chunks
    }
    
    # Step 3: Initialize your storage instances
    faiss_global_config = {
        "working_dir": "/tmp",
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    kv_global_config = {
        "working_dir": "/tmp",
        "llm_model_name": LLM_MODEL_NAME,
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    knowledge_graph = NetworkXStorage(namespace="nx_kg", global_config=faiss_global_config, embedding_func=openai_embed)  # Graph storage implementation
    entity_vector_db = FaissVectorDBStorage(namespace="faiss_entity", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for entities
    relationship_vector_db = FaissVectorDBStorage(namespace="faiss_relationship", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for relationships
    llm_cache = JsonKVStorage(namespace="llm_cache", global_config=kv_global_config, embedding_func=openai_embed)
    
    initialize_share_data()

    await knowledge_graph.initialize()
    await entity_vector_db.initialize()
    await relationship_vector_db.initialize()
    await llm_cache.initialize()  # if your cache supports/needs it

    # Step 4: Use the OpenAI LLM function from openai.py
    # Create a wrapper that matches the expected signature for extract_entities
    async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
        if history_messages is None:
            history_messages = []
    

        # Use Google GenAI
        return await gemini_complete_if_cache(
            model=LLM_MODEL_NAME,
            prompt=prompt,
            history_messages=history_messages,
            hashing_kv=llm_cache,
            temperature=0.2,
            max_tokens=max_tokens or 1024,
        )
    
    # Step 5: Configure the extraction process
    global_config = {
        "llm_model_func": llm_wrapper,  # Use our wrapper function
        "llm_model_name": LLM_MODEL_NAME,  # Required for some OpenAI functions
        "entity_extract_max_gleaning": max_gleaning,
        "force_llm_summary_on_merge": False,
        "llm_model_max_token_size": 1024,
        "summary_to_max_tokens": 256,
        "tiktoken_model_name": LLM_MODEL_NAME,
        "addon_params": {
            "language": "English",
            "entity_types": [
                "Software or Computational Method",
                "Scientific Method",
                "Infrastructure or Equipment",
                "Institution or Organization",
                "Ecological or Biological Entity",
                "Pollutant or Material",
                "Time Expression",
                "Health or Disease Concept",
                "Food or Nutrition Element",
                "Demographic Group",
                "Measurement or Quantity"
            ]
        },
    }
    
    # Step 6: Optional status tracking
    pipeline_status = {
        "latest_message": "",
        "history_messages": []
    }
    pipeline_status_lock = asyncio.Lock()
    
    # Step 7: Run the extraction
    await extract_entities(
        chunks=chunks,
        knowledge_graph_inst=knowledge_graph,
        entity_vdb=entity_vector_db,
        relationships_vdb=relationship_vector_db,
        global_config=global_config,
        pipeline_status=pipeline_status,
        pipeline_status_lock=pipeline_status_lock,
        llm_response_cache=llm_cache,
        cheatsheet_knowledge_graph_inst=cheatsheet_knowledge_graph_inst,
        write_result_to_txt=True,
        special_interest=CHEATSHEETS["special_interests"],
        prefix=prefix
    )
    
    return knowledge_graph


# Example usage in a Jupyter notebook
async def run_on_claim(content, nx_graph, max_gleaning, prefix=""):
    # monkeyReader = reader.MonkeyReader('monkey') # or 'lxml' or 'x2d'

    # read paper content
    # essay = monkeyReader.readEssay('./xmldata/Byjesh.pdf.tei.xml')
    # text = "\n".join(essay["Impact of fixed rise in temperature, CO 2 and change in rain fall"])
    text = content
    # nx_graph = nx.Graph()
    knowledge_graph = await process_text_to_knowledge_graph_on_claim(text, nx_graph,  max_gleaning, "example.txt", prefix=prefix)
    # Now you can query the knowledge graph for entities and relationships
    
    return knowledge_graph

In [3]:
async def process_text_to_knowledge_graph_on_reference(text_content,
                                          cheatsheet_knowledge_graph_inst,
                                          seed_entities,
                                          max_gleaning = 1,
                                          file_path="unknown_source",
                                          prefix=""
                                          ):
    # Step 1: Create chunks from your text content
    raw_chunks = chunking_by_token_size(
        content=text_content,
        split_by_character=None,  # Optional: specify a character to split by (e.g., "\n\n")
        split_by_character_only=False,
        overlap_token_size=128,
        max_token_size=1024,
        tiktoken_model=LLM_MODEL_NAME
    )
    
    # Step 2: Format chunks as required by extract_entities
    doc_id = compute_mdhash_id(text_content, prefix="doc-")
    chunks = {
        compute_mdhash_id(chunk["content"], prefix="chunk-"): {
            **chunk,
            "full_doc_id": doc_id,
            "file_path": file_path
        }
        for chunk in raw_chunks
    }
    
    # Step 3: Initialize your storage instances
    faiss_global_config = {
        "working_dir": "/tmp",
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    kv_global_config = {
        "working_dir": "/tmp",
        "llm_model_name": LLM_MODEL_NAME,
        "embedding_batch_num": 64,  # or another integer suitable for your setup
        "vector_db_storage_cls_kwargs": {
            "cosine_better_than_threshold": 0.2  # or another float threshold you want
        },
        "base_url": "https://api.openai.com/v1",
    }
    knowledge_graph = NetworkXStorage(namespace="nx_kg", global_config=faiss_global_config, embedding_func=openai_embed)  # Graph storage implementation
    entity_vector_db = FaissVectorDBStorage(namespace="faiss_entity", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for entities
    relationship_vector_db = FaissVectorDBStorage(namespace="faiss_relationship", global_config=faiss_global_config, embedding_func=openai_embed)  # Vector storage for relationships
    llm_cache = JsonKVStorage(namespace="llm_cache", global_config=kv_global_config, embedding_func=openai_embed)
    
    initialize_share_data()

    await knowledge_graph.initialize()
    await entity_vector_db.initialize()
    await relationship_vector_db.initialize()
    await llm_cache.initialize()  # if your cache supports/needs it

    # Step 4: Use the OpenAI LLM function from openai.py
    # Create a wrapper that matches the expected signature for extract_entities
    async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
        if history_messages is None:
            history_messages = []
    

        # Use Google GenAI
        return await gemini_complete_if_cache(
            model=LLM_MODEL_NAME,
            prompt=prompt,
            history_messages=history_messages,
            hashing_kv=llm_cache,
            temperature=0.2,
            max_tokens=max_tokens or 1024,
        )
    
    # Step 5: Configure the extraction process
    global_config = {
        "llm_model_func": llm_wrapper,  # Use our wrapper function
        "llm_model_name": LLM_MODEL_NAME,  # Required for some OpenAI functions
        "entity_extract_max_gleaning": max_gleaning,
        "force_llm_summary_on_merge": False,
        "llm_model_max_token_size": 1024,
        "summary_to_max_tokens": 256,
        "tiktoken_model_name": LLM_MODEL_NAME,
        "addon_params": {
            "language": "English",
            "entity_types": [
                "Software or Computational Method",
                "Scientific Method",
                "Infrastructure or Equipment",
                "Institution or Organization",
                "Ecological or Biological Entity",
                "Pollutant or Material",
                "Time Expression",
                "Health or Disease Concept",
                "Food or Nutrition Element",
                "Demographic Group",
                "Measurement or Quantity"
            ]
        },
    }
    
    # Step 6: Optional status tracking
    pipeline_status = {
        "latest_message": "",
        "history_messages": []
    }
    pipeline_status_lock = asyncio.Lock()
    
    # Step 7: Run the extraction
    await extract_evidence(
        chunks=chunks,
        knowledge_graph_inst=knowledge_graph,
        entity_vdb=entity_vector_db,
        relationships_vdb=relationship_vector_db,
        global_config=global_config,
        seed_entities=seed_entities,
        pipeline_status=pipeline_status,
        pipeline_status_lock=pipeline_status_lock,
        llm_response_cache=llm_cache,
        cheatsheet_knowledge_graph_inst=cheatsheet_knowledge_graph_inst,
        write_result_to_txt=True,
        special_interest=CHEATSHEETS["special_interests"],
        prefix=prefix
    )
    
    return knowledge_graph


# Example usage in a Jupyter notebook
async def run_on_reference(content, nx_graph, seed_entities, max_gleaning, prefix=""):
    # monkeyReader = reader.MonkeyReader('monkey') # or 'lxml' or 'x2d'

    # read paper content
    # essay = monkeyReader.readEssay('./xmldata/Byjesh.pdf.tei.xml')
    # text = "\n".join(essay["Impact of fixed rise in temperature, CO 2 and change in rain fall"])
    text = content
    # nx_graph = nx.Graph()
    knowledge_graph = await process_text_to_knowledge_graph_on_reference(text,
                                                                         nx_graph,
                                                                         seed_entities,
                                                                         max_gleaning,
                                                                         "example.txt",
                                                                         prefix=prefix)
    # Now you can query the knowledge graph for entities and relationships
    
    return knowledge_graph

In [4]:
import networkx as nx
from utils import get_latest_result, read_knowledge_graph_from_pickle

def get_othology_graph(entity_nodes, entity_edges):
    G = nx.Graph()
    for edge_keys, edge_list in entity_edges.items():
        for edge in edge_list:
            source = edge.get("src_id", "")
            target = edge.get("tgt_id", "")
            if source and target:
                if source in entity_nodes and target in entity_nodes:
                    G.add_edge(
                        entity_nodes[source][0]["entity_type"],
                        entity_nodes[target][0]["entity_type"],
                    )
    return G

In [ ]:
from utils import get_latest_result
batch2_data = dataset_loader["subtask1_batch2"]
index = 0
claim_content, reference_content = batch2_data.at[index, "claim"], batch2_data.at[index, "reference"]

_ = await run_on_claim(claim_content, nx.Graph(), 0, prefix=f"claim_batch2_{index}")

In [5]:
from utils import get_latest_result
from tqdm import trange
batch2_data = dataset_loader["subtask1_batch2"]
for index in trange(100, 1000):
    claim_content = batch2_data.at[index, "claim"]

    _ = await run_on_claim(claim_content, nx.Graph(),
                           0, prefix=f"claim_batch2_{index}")

  0%|          | 0/900 [00:00<?, ?it/s]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 77927 Shared-Data created for Single Process
INFO: Process 77927 initialized updated flags for namespace: [nx_kg]
INFO: Process 77927 initialized updated flags for namespace: [faiss_entity]
INFO: Process 77927 initialized updated flags for namespace: [faiss_relationship]
INFO: Process 77927 initialized updated flags for namespace: [llm_cache]
INFO: Process 77927 ready to initialize storage namespace: [llm_cache]
/home/com3dian/Github/meta-knowledge-harvesting-llm/kaggle/operationCheatSheet.py:730: RuntimeWarning: coroutine 'extract_entities.<locals>.nightly_kg_to_text' was never awaited
  hint_prompt = fill_nightly_prompt.format(
  0%|          | 1/900 [00:24<6:08:45, 24.61s/it]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 77927 Shared-Data already initialized 

Final result:
("entity"<|>"Collaborative Navigation"<|>"Collaborative Navigation"<|>"Scientific Method"<|>"Robots can assist users in navigation tasks, predicting users' motion intentions to reduce workload and improve performance [7]."<|>"Collaborative Navigation"<|>"N/A"<|>"N/A")##
("entity"<|>"Robots"<|>"Robots"<|>"Infrastructure or Equipment"<|>"Robots can assist users in navigation tasks, predicting users' motion intentions to reduce workload and improve performance [7]."<|>"Robots"<|>"N/A"<|>"N/A")##
("entity"<|>"Users"<|>"Users"<|>"Demographic Group"<|>"Robots can assist users in navigation tasks, predicting users' motion intentions to reduce workload and improve performance [7]."<|>"users"<|>"N/A"<|>"N/A")##
("relationship"<|>"Robots"<|>"Users"<|>"Robots can assist users in navigation tasks, predicting users' motion intentions to reduce workload and improve performance [7]."<|>"assistance, human-robot interaction"<|>9)##
("relationship"<|>"Collaborative Navigation"<|>"Robots"<|

  0%|          | 2/900 [00:47<5:49:57, 23.38s/it]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 77927 Shared-Data already initialized (multiprocess=False)
INFO: Process 77927 storage namespace already initialized: [llm_cache]


Final result:
("entity"<|>"Distributed Control"<|>"Scientific Method"<|>"Robots cannot perform tasks like forming geometric patterns or antenna arrays without a central supervisor, as Robots require constant guidance and real-time adjustments from a central system."<|>"Distributed Control"<|><Nightly Inference><|><Nightly Inference>)##
("entity"<|>"Robots"<|>"Infrastructure or Equipment"<|>"Robots cannot perform tasks like forming geometric patterns or antenna arrays without a central supervisor, as Robots require constant guidance and real-time adjustments from a central system."<|>"Robots"<|><Nightly Inference><|><Nightly Inference>)##
("entity"<|>"Central System"<|>"Software or Computational Method"<|>"Robots cannot perform tasks like forming geometric patterns or antenna arrays without a central supervisor, as Robots require constant guidance and real-time adjustments from a central system."<|>"central supervisor, central system"<|><Nightly Inference><|><Nightly Inference>)##
("rel

  0%|          | 3/900 [01:01<4:46:30, 19.16s/it]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 77927 Shared-Data already initialized (multiprocess=False)
INFO: Process 77927 storage namespace already initialized: [llm_cache]


...

Final result:
("entity"<|>"Non-native Species"<|>"Non-native Species"<|>"Ecological or Biological Entity"<|>"Offshore wind farms can facilitate the spread of non-native species, non-native species can disrupt local ecosystems."<|>"Non-native Species"<|><|>)##
("entity"<|>"Offshore Wind Farms"<|>"Offshore Wind Farms"<|>"Infrastructure or Equipment"<|>"Offshore wind farms can facilitate the spread of non-native species, non-native species can disrupt local ecosystems."<|>"Offshore wind farms"<|><|>)##
("entity"<|>"Local Ecosystems"<|>"Local Ecosystems"<|>"Ecological or Biological Entity"<|>"non-native species can disrupt local ecosystems."<|>"local ecosystems"<|><|>)##
("relationship"<|>"Offshore Wind Farms"<|>"Non-native Species"<|>"Offshore wind farms can facilitate the spread of non-native species."<|>"environmental impact, species introduction"<|>8)##
("relationship"<|>"Non-native Species"<|>"Local Ecosystems"<|>"non-native species can disrupt local ecosystems."<|>"ecological impact,

100%|█████████▉| 899/900 [8:00:54<00:32, 32.74s/it]No existing Faiss index file found. Starting fresh.
No existing Faiss index file found. Starting fresh.
INFO: Process 77927 Shared-Data already initialized (multiprocess=False)
INFO: Process 77927 storage namespace already initialized: [llm_cache]


Final result:
("entity"<|>"Wind Farms"<|>"Wind Farms"<|>"Infrastructure or Equipment"<|>"Placing wind farms in areas with high recreational activities and near sensitive marine habitats can increase negative impacts."<|>"wind farms"<|>"0"<|>"0")##
("entity"<|>"Sensitive Marine Habitats"<|>"Sensitive Marine Habitats"<|>"Ecological or Biological Entity"<|>"Placing wind farms in areas with high recreational activities and near sensitive marine habitats can increase negative impacts."<|>"sensitive marine habitats"<|>"0"<|>"0")##
("entity"<|>"Negative Impacts"<|>"Negative Impacts"<|>"Health or Disease Concept"<|>"Placing wind farms in areas with high recreational activities and near sensitive marine habitats can increase negative impacts."<|>"negative impacts"<|>"0"<|>"0")##
("entity"<|>"Recreational Activities"<|>"Recreational Activities"<|>"Health or Disease Concept"<|>"Placing wind farms in areas with high recreational activities and near sensitive marine habitats can increase negative i

100%|██████████| 900/900 [8:01:15<00:00, 32.08s/it]

Final result:
("entity"<|>"Lead"<|>"Lead"<|>"Pollutant or Material"<|>"Geological faults and radon emissions are the primary causes of lead levels in the soil, suggesting that all lead contamination is due to these natural sources [6]."<|>"lead levels in the soil"<|><|>)##
("entity"<|>"Geological faults"<|>"Geological faults"<|>"Infrastructure or Equipment"<|>"Geological faults and radon emissions are the primary causes of lead levels in the soil, suggesting that all lead contamination is due to these natural sources [6]."<|>"Geological faults"<|><|>)##
("entity"<|>"Radon emissions"<|>"Radon emissions"<|>"Pollutant or Material"<|>"Geological faults and radon emissions are the primary causes of lead levels in the soil, suggesting that all lead contamination is due to these natural sources [6]."<|>"radon emissions"<|><|>)##
("relationship"<|>"Geological faults"<|>"Lead"<|>"Geological faults and radon emissions are the primary causes of lead levels in the soil, suggesting that all lead co

In [ ]:
from utils import get_latest_result_by_prefix
from tqdm import trange

batch2_data = dataset_loader["subtask1_batch2"]
for index in trange(100):
    reference_content = batch2_data.at[index, "reference"]
    claim_nodes, claim_edges = get_latest_result_by_prefix(f"claim_batch2_{index}")
    claim_ontology_graph = get_othology_graph(claim_nodes, claim_edges)
    seed_entities = list(claim_nodes.keys())

    _ = await run_on_reference(reference_content, claim_ontology_graph, seed_entities, 0, prefix=f"reference_batch2_{index}")

In [25]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import numpy as np

# # Plot distributions for each row
# plt.figure(figsize=(12, 6))

def get_src_tgt_dict(similarity_matrix, claim_node_list, reference_node_list):
    src_tgt_dict = dict()
    for i, row in enumerate(similarity_matrix):
        # Fit KMeans with 2 clusters
        kmeans = KMeans(n_clusters=2, random_state=0, n_init=10)
        row_reshaped = row.reshape(-1, 1)
        kmeans.fit(row_reshaped)
        centers = np.sort(kmeans.cluster_centers_.flatten())
        threshold = np.mean(centers)  # Midpoint between the two centers
        threshold = max(threshold, 0.25)

        # Find values above threshold
        anomalies = row[row > threshold]
        src_tgt_dict[claim_node_list[i]] = []
        for index, sim in enumerate(row > threshold):
            if sim:
                src_tgt_dict[claim_node_list[i]].append(reference_node_list[index])

    return src_tgt_dict

# for i, row in enumerate(similarity_matrix):
#     # Fit KMeans with 2 clusters
#     kmeans = KMeans(n_clusters=2, random_state=0, n_init=10)
#     row_reshaped = row.reshape(-1, 1)
#     kmeans.fit(row_reshaped)
#     centers = np.sort(kmeans.cluster_centers_.flatten())
#     threshold = np.mean(centers)  # Midpoint between the two centers
#     threshold = max(threshold, 0.25)

#     # Find values above threshold
#     anomalies = row[row > threshold]
#     src_tgt_dict[claim_node_list[i]] = []
#     for index, sim in enumerate(row > threshold):
#         if sim:
#             src_tgt_dict[claim_node_list[i]].append(reference_node_list[index])


#     # Plot distribution
#     sns.kdeplot(row, label=f'Row {i}', fill=True, alpha=0.5)
#     # Plot threshold
#     plt.axvline(threshold, color=f'C{i}', linestyle='--', label=f'Row {i} KMeans threshold: {threshold:.2f}')
#     # Plot anomalies
#     plt.scatter(anomalies, [0.01]*len(anomalies), color=f'C{i}', marker='o', s=100, edgecolor='k', label=f'Row {i} anomalies' if len(anomalies) > 0 else "")

#     # Optionally, print anomaly values
#     if len(anomalies) > 0:
#         print(f"Row {i} anomalies (>{threshold:.2f}): {anomalies}")

# plt.xlabel('Value')
# plt.ylabel('Density')
# plt.title('Distribution with 3-Sigma Anomaly Threshold')
# plt.legend()
# plt.show()

In [ ]:
from kg_compare import compare_knowledge_graphs_similarity
import seaborn as sns

import numpy as np
from sklearn.cluster import KMeans
all_responses = []

for index in range(3):
    if index % 10 == 0:
        print(f"Processing index {index}")

    
    claim_nodes, claim_edges = get_latest_result_by_prefix(f"claim_test_{index}_")
    reference_nodes, reference_edges = get_latest_result_by_prefix(f"reference_test_{index}_")
    
    claim_content = batch2_data.at[index, "claim"]
    reference_content = batch2_data.at[index, "reference"]

    claim_node_list = list(claim_nodes.keys())
    reference_node_list = list(reference_nodes.keys())
    src_tgt_dict = dict()

    if claim_node_list == [] or reference_node_list == []:
        all_responses.append("Unverifiable")
        continue

    similarity_matrix = compare_knowledge_graphs_similarity(claim_node_list,
                                                            reference_node_list,
                                                            if_plot=False)
    src_tgt_dict = get_src_tgt_dict(similarity_matrix, claim_node_list, reference_node_list)
    evidence_list = []
    claim_reference_edge_map = get_map_src_tgt_dict(src_tgt_dict, claim_edges)
    for claim_edge_key, reference_edge_list in claim_reference_edge_map.items():
        evidence_edges = set(reference_edge_list) & set(reference_edges.keys())

        for evidence_edge in evidence_edges:
            reference_edge = reference_edges[evidence_edge]
            best_match, _ = find_best_span(reference_edge[0]["description"], reference_content)
            evidence_list.append(best_match)

    # if set(evidence_list) == set():
    #     all_responses.append("Unverifiable")
    #     continue

    # evidence_text = " ".join(set(evidence_list))

    # classification_prompt = f"""
    # You are a helpful assistant that classifies the claim into one of the following categories, based on the evidence:
    # - Entailment
    # - Contradiction
    # - Unverifiable

    # The claim is:
    # {claim_content}

    # The evidence is:
    # {evidence_text} 

    # Instructions:
    # - You MUST respond with ONLY one of these three words: "Entailment", "Contradiction", or "Unverifiable"
    # - If the evidence is not related to the claim, you MUST respond with "Unverifiable"
    # - If the claim is misinterprets, misleads, or overstates the evidence, you must respond with "Contradiction"
    # - If the claim is supported by the evidence, you must respond with "Entailment"
    # - Do not provide any explanation, reasoning, or additional text
    # - Your response must be exactly one of the three categories above
    # """

    # response = await llm_wrapper(classification_prompt)
    # all_responses.append(response)
    
    

In [ ]:
from nltk.tokenize import sent_tokenize
from rapidfuzz import fuzz

from kg_compare import get_src_tgt_dict, get_map_src_tgt_dict, compare_knowledge_graphs_similarity

index = 0
claim_nodes, claim_edges = get_latest_result_by_prefix(f"claim_test_{index}_")
reference_nodes, reference_edges = get_latest_result_by_prefix(f"reference_test_{index}_")
reference_content = batch2_data.at[index, "reference"]

claim_node_list = list(claim_nodes.keys())
reference_node_list = list(reference_nodes.keys())

similarity_matrix = compare_knowledge_graphs_similarity(claim_node_list,
                                                        reference_node_list,
                                                        if_plot=False)
src_tgt_dict = get_src_tgt_dict(similarity_matrix, claim_node_list, reference_node_list)

claim_reference_edge_map = get_map_src_tgt_dict(src_tgt_dict, claim_edges)

def find_best_span_for_all_evidences(reference_edges,
                                     claim_reference_edge_map,
                                     paragraph,
                                     max_window_size=5):
    sentences = sent_tokenize(paragraph)
    best_span_list = []
    for claim_edge_key, reference_edge_list in claim_reference_edge_map.items():
        evidence_edges = set(reference_edge_list) & set(reference_edges.keys())
        print(evidence_edges)
        for evidence_edge in evidence_edges:
            reference_edge = reference_edges[evidence_edge]

            best_score = -1
            best_span = None

            for window_size in range(1, min(max_window_size, len(sentences)) + 1):
                for i in range(len(sentences) - window_size + 1):
                    window = ' '.join(sentences[i:i+window_size])
                    score = fuzz.ratio(reference_edge[0]["description"], window)
                    if score > best_score:
                        best_score = score
                        best_span = (i, i+window_size)
            best_span_list.append(best_span)
    
    return best_span_list


find_best_span_for_all_evidences(reference_edges,
                                 claim_reference_edge_map,
                                 reference_content)


In [ ]:
claim_reference_edge_map[('Digital Transformation', 'ICT')]

In [ ]:
from rapidfuzz import fuzz, process

import nltk
# nltk.download('punkt_tab')
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from rapidfuzz import fuzz

def find_best_span(target_chunk, paragraph, max_window_size=5):
    sentences = sent_tokenize(paragraph)
    best_score = -1
    best_span = None

    for window_size in range(1, min(max_window_size, len(sentences)) + 1):
        for i in range(len(sentences) - window_size + 1):
            window = ' '.join(sentences[i:i+window_size])
            score = fuzz.ratio(target_chunk, window)
            if score > best_score:
                best_score = score
                best_span = window

    return best_span, best_score


evidence_list = []
claim_reference_edge_map = get_map_stc_tgt_dict(src_tgt_dict)
for claim_edge_key, reference_edge_list in claim_reference_edge_map.items():
    evidence_edges= set(reference_edge_list) & set(reference_edges.keys())
    for evidence_edge in evidence_edges:
        reference_edge = reference_edges[evidence_edge]
        best_match, _ = find_best_span(reference_edge[0]["description"], reference_content)
        evidence_list.append(best_match)

evidence_text = " ".join(set(evidence_list))
classification_prompt = f"""
You are a helpful assistant that classifies the claim into one of the following categories, based on the evidence:
- Entailment
- Contradiction
- Unverifiable

The claim is:
{claim_content}

The evidence is:
{evidence_text}

Instructions:
- You MUST respond with ONLY one of these three words: "Entailment", "Contradiction", or "Unverifiable"
- Do not provide any explanation, reasoning, or additional text
- Your response must be exactly one of the three categories above
"""
print(classification_prompt)

In [ ]:
from utils import get_latest_result_by_prefix
from kg_compare import compare_knowledge_graphs_similarity, get_src_tgt_dict, find_best_span

index = 0

claim_nodes, claim_edges = get_latest_result_by_prefix(f"claim_batch2_{index}_")
reference_nodes, reference_edges = get_latest_result_by_prefix(f"reference_batch2_{index}_")

batch2_data = dataset_loader["subtask1_batch2"]

claim_content = batch2_data.at[index, "claim"]
reference_content = batch2_data.at[index, "reference"]

claim_node_list = list(claim_nodes.keys())
reference_node_list = list(reference_nodes.keys())
reference_edge_list = list(reference_edges.keys())
src_tgt_dict = dict()

similarity_matrix = compare_knowledge_graphs_similarity(claim_node_list,
                                                        reference_node_list,
                                                        if_plot=False)
src_tgt_dict = get_src_tgt_dict(similarity_matrix,
                                claim_node_list,
                                reference_node_list)
evidence_list = []

claim_reference_edge_map = get_map_src_tgt_dict(src_tgt_dict, claim_edges)
for claim_edge_key, mapped_reference_edge_list in claim_reference_edge_map.items():
    evidence_edges = set(reference_edge_list) & set(mapped_reference_edge_list)
    for evidence_edge in evidence_edges:
        reference_edge = reference_edges[evidence_edge]
        best_match, _ = find_best_span(reference_edge[0]["description"], reference_content)
        evidence_list.append(best_match)

In [ ]:
evidence_list

In [ ]:
src_tgt_dict
import itertools

def get_map_src_tgt_dict(src_tgt_dict, claim_edges):
    combinations = list(itertools.product(src_tgt_dict.keys(), repeat=2))
    ans = dict()
    for key_pair in claim_edges.keys():
        tgt_list_1 = src_tgt_dict[key_pair[0]]
        tgt_list_2 = src_tgt_dict[key_pair[1]]
        combinations = list(itertools.product(tgt_list_1, tgt_list_2)) + list(itertools.product(tgt_list_2, tgt_list_1))
        ans[key_pair] = combinations
    return ans


get_map_src_tgt_dict(src_tgt_dict, claim_edges)

In [ ]:
import itertools



# Get cartesian product of list with itself
combinations = list(itertools.product(src_tgt_dict.keys(), repeat=2))
print(combinations)

In [ ]:
from json_kv_iml import JsonKVStorage
from dotenv import load_dotenv
from operationCheatSheet import chunking_by_token_size, extract_entities, extract_evidence
from utils import compute_mdhash_id
from geminillm import gemini_complete_if_cache
from openaillm import openai_embed, openai_complete
from faiss_impl import FaissVectorDBStorage
from networkx_impl import NetworkXStorage
from json_kv_iml import JsonKVStorage
from shared_storage import initialize_share_data
from cheatsheet import CHEATSHEETS

LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20"

kv_global_config = {
    "working_dir": "/tmp",
    "llm_model_name": LLM_MODEL_NAME,
    "embedding_batch_num": 64,  # or another integer suitable for your setup
    "vector_db_storage_cls_kwargs": {
        "cosine_better_than_threshold": 0.2  # or another float threshold you want
    },
    "base_url": "https://api.openai.com/v1",
}
llm_cache = JsonKVStorage(namespace="llm_cache",
                          global_config=kv_global_config,
                          embedding_func=openai_embed)

initialize_share_data()

await llm_cache.initialize()


async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
    if history_messages is None:
        history_messages = []


    # Use Google GenAI
    return await gemini_complete_if_cache(
        model=LLM_MODEL_NAME,
        prompt=prompt,
        history_messages=history_messages,
        hashing_kv=llm_cache,
        temperature=0.2,
        max_tokens=max_tokens or 1024,
    )


In [63]:
results = []
for index, row in batch2_data.iterrows():
    if index >= 100:
        break
    row_id = row.ID
    true_label = row.label
    response = all_responses[index]
    if response.lower().startswith("entailment"):
        predicted_label = "entail"
    elif response.lower().startswith("contradiction"):
        predicted_label = "contra"
    elif response.lower().startswith("unverifiable"):
        predicted_label = "unver"
    else:
        raise ValueError(f"Invalid response: {response}")
    results.append({
        'ID': row_id,
        'true_label': true_label,
        'predicted_label': predicted_label,
        'batch': 'batch2'
    })

results_df = pd.DataFrame(results)

In [ ]:
results

In [5]:
def create_knowledge_graph_from_results(nodes, edges) -> nx.Graph:
    """
    Creates a NetworkX graph from the extracted nodes and edges.
    Returns:
        nx.Graph: Graph containing entities as nodes and their relationships
    """
    G = nx.Graph()
    
    # Add nodes from the extracted results
    for entity_name, entity_list in nodes.items():
        for entity in entity_list:
            G.add_node(
                entity["entity_name"],
                entity_type=entity.get("entity_type", ""),
                description=entity.get("description", ""),
                source_id=entity.get("source_id", ""),
                file_path=entity.get("file_path", "")
            )
    
    # Add edges from the extracted results
    for edge_key, edge_list in edges.items():
        for edge in edge_list:
            source = edge.get("src_id", "")
            target = edge.get("tgt_id", "")
            if source and target:
                G.add_edge(
                    source,
                    target,
                    description=edge.get("description", ""),
                    keywords=edge.get("keywords", ""),
                    weight=edge.get("weight", 1.0),
                    source_id=edge.get("source_id", ""),
                    file_path=edge.get("file_path", "")
                )
    
    return G

def compare_knowledge_graphs(claim_graph, reference_graph):
    """
    Compare claim graph with reference graph to determine entailment relationship.
    
    Args:
        claim_graph: NetworkX graph from claim
        reference_graph: NetworkX graph from reference
    
    Returns:
        str: "Entailment", "Contradiction", or "Unverifiable"
    """
    # Get nodes from both graphs
    claim_nodes = set(claim_graph.nodes())
    reference_nodes = set(reference_graph.nodes())
    
    # Check if all claim nodes exist in reference graph
    nodes_match = claim_nodes.issubset(reference_nodes)
    
    if not nodes_match:
        # If nodes don't match, it's unverifiable
        return "unver"
    
    # If nodes match, check edges with their connected nodes
    claim_edges_with_nodes = set()
    reference_edges_with_nodes = set()
    
    # Extract edges with their node information for claim graph
    for edge in claim_graph.edges():
        source, target = edge
        # Create a tuple that represents the edge with its nodes
        claim_edges_with_nodes.add((source, target))
        # Also add the reverse direction to handle undirected graphs
        claim_edges_with_nodes.add((target, source))
    
    # Extract edges with their node information for reference graph
    for edge in reference_graph.edges():
        source, target = edge
        # Create a tuple that represents the edge with its nodes
        reference_edges_with_nodes.add((source, target))
        # Also add the reverse direction to handle undirected graphs
        reference_edges_with_nodes.add((target, source))
    
    # Check if all claim edges (with their nodes) exist in reference graph
    edges_match = all(edge in reference_edges_with_nodes for edge in claim_edges_with_nodes)
    
    if edges_match:
        # Both nodes and edges match - entailment
        return "entail"
    else:
        # Nodes match but edges don't - contradiction
        return "contra"


In [ ]:
from utils import get_latest_result, read_knowledge_graph_from_pickle
from sklearn.metrics import f1_score, classification_report
import pandas as pd

# Process all rows in batch2 and batch3
results = []

# Process batch2
print("Processing batch2...")
batch2_data = dataset_loader["subtask1_batch2"]
for idx, row in batch2_data.iterrows():
    try:
        reference_content = row.reference
        claim_content = row.claim
        row_id = row.ID
        true_label = row.label

        # Create knowledge graph from claim
        _ = await main(claim_content, nx.Graph(), 0)
        nodes_claim, edges_claim = get_latest_result()
        othology_graph_from_claim = get_othology_graph(nodes_claim, edges_claim)
        knowledge_graph_from_claim = create_knowledge_graph_from_results(nodes_claim, edges_claim)
        
        # Create knowledge graph from reference
        _ = await main(reference_content, othology_graph_from_claim, 1)
        nodes_reference, edges_reference = get_latest_result()
        knowledge_graph_from_reference = create_knowledge_graph_from_results(nodes_reference, edges_reference)
        
        # Compare the two knowledge graphs
        predicted_label = compare_knowledge_graphs(knowledge_graph_from_claim, knowledge_graph_from_reference)
        
        results.append({
            'ID': row_id,
            'true_label': true_label,
            'predicted_label': predicted_label,
            'batch': 'batch2'
        })
        
        print(f"Processed {row_id}: True={true_label}, Predicted={predicted_label}")
        
    except Exception as e:
        print(f"Error processing row {row_id}: {e}")
        results.append({
            'ID': row_id,
            'true_label': true_label,
            'predicted_label': 'unver',  # Default to unverifiable on error
            'batch': 'batch2'
        })

# Process batch3 if available
if "subtask1_batch3" in dataset_loader:
    print("\nProcessing batch3...")
    batch3_data = dataset_loader["subtask1_batch3"]
    for idx, row in batch3_data.iterrows():
        try:
            reference_content = row.reference
            claim_content = row.claim
            row_id = row.ID
            true_label = row.label
            
            # Create knowledge graph from reference
            knowledge_graph_from_reference = await main(reference_content, nx.Graph())
            nodes_reference, edges_reference = get_latest_result()
            knowledge_graph_from_reference = create_knowledge_graph_from_results(nodes_reference, edges_reference)
            
            # Create knowledge graph from claim
            knowledge_graph_from_claim = await main(claim_content, knowledge_graph_from_reference)
            nodes_claim, edges_claim = get_latest_result()
            knowledge_graph_from_claim = create_knowledge_graph_from_results(nodes_claim, edges_claim)
            
            # Compare the two knowledge graphs
            predicted_label = compare_knowledge_graphs(knowledge_graph_from_claim, knowledge_graph_from_reference)
            
            results.append({
                'ID': row_id,
                'true_label': true_label,
                'predicted_label': predicted_label,
                'batch': 'batch3'
            })
            
            print(f"Processed {row_id}: True={true_label}, Predicted={predicted_label}")
            
        except Exception as e:
            print(f"Error processing row {row_id}: {e}")
            results.append({
                'ID': row_id,
                'true_label': true_label,
                'predicted_label': 'unver',  # Default to unverifiable on error
                'batch': 'batch3'
            })

# Convert results to DataFrame
results_df = pd.DataFrame(results)


In [ ]:
# Save results_df to local file
import os
from datetime import datetime

# Create output directory if it doesn't exist
output_dir = "outputs/kaggle_results"
os.makedirs(output_dir, exist_ok=True)

# Generate timestamp for filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Save as CSV
csv_filename = f"kaggle_evaluation_results_{timestamp}.csv"
csv_filepath = os.path.join(output_dir, csv_filename)
results_df.to_csv(csv_filepath, index=False)
print(f"Results saved to CSV: {csv_filepath}")

# Save as JSON for better readability
json_filename = f"kaggle_evaluation_results_{timestamp}.json"
json_filepath = os.path.join(output_dir, json_filename)
results_df.to_json(json_filepath, orient='records', indent=2)
print(f"Results saved to JSON: {json_filepath}")

# Save as pickle for preserving data types
pickle_filename = f"kaggle_evaluation_results_{timestamp}.pkl"
pickle_filepath = os.path.join(output_dir, pickle_filename)
results_df.to_pickle(pickle_filepath)
print(f"Results saved to Pickle: {pickle_filepath}")


In [ ]:
from sklearn.metrics import f1_score, classification_report
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Calculate evaluation metrics
true_labels = results_df['true_label'].tolist()
predicted_labels = results_df['predicted_label'].tolist()

# Calculate weighted F1 score (main evaluation metric)
weighted_f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"\n=== EVALUATION RESULTS ===")
print(f"Total samples processed: {len(results_df)}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")

# Calculate accuracy
accuracy = (results_df['true_label'] == results_df['predicted_label']).mean()
print(f"Accuracy: {accuracy:.4f}")

# Print detailed classification report
print(f"\n=== DETAILED CLASSIFICATION REPORT ===")
print(classification_report(true_labels, predicted_labels))

# Show results by batch
print(f"\n=== RESULTS BY BATCH ===")
for batch in results_df['batch'].unique():
    batch_results = results_df[results_df['batch'] == batch]
    batch_accuracy = (batch_results['true_label'] == batch_results['predicted_label']).mean()
    batch_f1 = f1_score(batch_results['true_label'], batch_results['predicted_label'], average='weighted')
    print(f"{batch}: Accuracy={batch_accuracy:.4f}, Weighted F1={batch_f1:.4f}")

# Calculate F1 scores for batch2 and batch3 separately
print(f"\n=== INDIVIDUAL BATCH F1 SCORES ===")
if 'batch2' in results_df['batch'].values:
    batch2_results = results_df[results_df['batch'] == 'batch2']
    batch2_f1 = f1_score(batch2_results['true_label'], batch2_results['predicted_label'], average='weighted')
    print(f"Batch2 Weighted F1 Score: {batch2_f1:.4f}")

if 'batch3' in results_df['batch'].values:
    batch3_results = results_df[results_df['batch'] == 'batch3']
    batch3_f1 = f1_score(batch3_results['true_label'], batch3_results['predicted_label'], average='weighted')
    print(f"Batch3 Weighted F1 Score: {batch3_f1:.4f}")

# Display first few results
print(f"\n=== SAMPLE RESULTS ===")
print(results_df.head(10))


In [ ]:
all_responses

In [ ]:
# Print detailed comparison information
print(f"\nClaim graph nodes: {list(knowledge_graph_from_claim.nodes())}")
print(f"Reference graph nodes: {list(knowledge_graph_from_reference.nodes())}")
print(f"Claim graph edges: {list(knowledge_graph_from_claim.edges())}")
print(f"Reference graph edges: {list(knowledge_graph_from_reference.edges())}")

# Print node matching details
claim_nodes = set(knowledge_graph_from_claim.nodes())
reference_nodes = set(knowledge_graph_from_reference.nodes())
missing_nodes = claim_nodes - reference_nodes
if missing_nodes:
    print(f"Missing nodes in reference: {missing_nodes}")
else:
    print("All claim nodes found in reference graph")

# Print edge matching details
claim_edges = set(knowledge_graph_from_claim.edges())
reference_edges = set(knowledge_graph_from_reference.edges())
missing_edges = claim_edges - reference_edges
if missing_edges:
    print(f"Missing edges in reference: {missing_edges}")
else:
    print("All claim edges found in reference graph")
